In [1]:
import gc
import gzip
import itertools
import time
import warnings
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import seaborn as sns
import torch
from pytorch_optimizer import MADGRAD
from category_encoders import CountEncoder
from deepctr_torch.inputs import DenseFeat, SparseFeat, get_feature_names
from deepctr_torch.models import AFM, DCN, DIFM, AutoInt, DeepFM, xDeepFM
from deepctr_torch.callbacks import EarlyStopping, ModelCheckpoint
from matplotlib import pyplot as plt
from sklearn import base
from sklearn.metrics import confusion_matrix, log_loss, roc_auc_score
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm
from typing_extensions import Self

warnings.filterwarnings("ignore")


2024-05-13 16:00:54.358799: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-13 16:00:54.359066: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-13 16:00:54.362713: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-13 16:00:54.393230: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-13 16:00:54.940027: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

In [2]:
NAN_INT = 7535805


class LabelEncoder(base.BaseEstimator):
    """Label Encoder that groups infrequent values into one label.

    Attributes:
        min_obs (int): minimum number of observation to assign a label.
        label_encoders (list of dict): label encoders for columns
        label_maxes (list of int): maximum of labels for columns
    """

    def __init__(self, min_obs: int = 10):
        """Initialize the OneHotEncoder class object.

        Args:
            min_obs (int): minimum number of observation to assign a label.
        """

        self.min_obs = min_obs
        self.is_fitted = False

    def __repr__(self):
        return ("LabelEncoder(min_obs={})").format(self.min_obs)

    def _get_label_encoder_and_max(self, x: pd.Series) -> tuple[dict, int]:
        """Return a mapping from values and its maximum of a column to integer labels.

        Args:
            x (pandas.Series): a categorical column to encode.

        Returns:
            (tuple):
                - (dict): mapping from values of features to integers
                - (int): maximum label
        """

        # NaN cannot be used as a key for dict. Impute it with a random
        # integer.
        label_count = x.fillna(NAN_INT).value_counts()
        n_uniq = label_count.shape[0]

        label_count = label_count[label_count >= self.min_obs]
        n_uniq_new = label_count.shape[0]

        # If every label appears more than min_obs, new label starts from 0.
        # Otherwise, new label starts from 1 and 0 is used for all old labels
        # that appear less than min_obs.
        offset = 0 if n_uniq == n_uniq_new else 1

        label_encoder = pd.Series(np.arange(n_uniq_new) + offset, index=label_count.index)
        max_label = label_encoder.max()
        label_encoder = label_encoder.to_dict()

        return label_encoder, max_label

    def _transform_col(self, x: pd.Series, i: int) -> pd.Series:
        """Encode one categorical column into labels.

        Args:
            x (pandas.Series): a categorical column to encode
            i (int): column index

        Returns:
            (pandas.Series): a column with labels.
        """
        return x.fillna(NAN_INT).map(self.label_encoders[i]).fillna(0).astype(int)

    def fit(self, X: pd.DataFrame, y: pd.Series | None = None) -> Self:
        self.label_encoders = [None] * X.shape[1]
        self.label_maxes = [None] * X.shape[1]

        for i, col in enumerate(X.columns):
            (
                self.label_encoders[i],
                self.label_maxes[i],
            ) = self._get_label_encoder_and_max(X[col])

        self.is_fitted = True
        return self

    def transform(self, X: pd.DataFrame) -> pd.DataFrame:
        """Encode categorical columns into label encoded columns

        Args:
            X (pandas.DataFrame): categorical columns to encode

        Returns:
            (pandas.DataFrame): label encoded columns
        """

        assert self.is_fitted, "fit() or fit_transform() must be called before transform()."

        X = X.copy()
        for i, col in enumerate(X.columns):
            X.loc[:, col] = self._transform_col(X[col], i)

        return X

    def fit_transform(self, X: pd.DataFrame, y: pd.Series | None = None) -> pd.DataFrame:
        """Encode categorical columns into label encoded columns

        Args:
            X (pandas.DataFrame): categorical columns to encode

        Returns:
            (pandas.DataFrame): label encoded columns
        """

        self.label_encoders = [None] * X.shape[1]
        self.label_maxes = [None] * X.shape[1]

        X = X.copy()
        for i, col in enumerate(X.columns):
            (
                self.label_encoders[i],
                self.label_maxes[i],
            ) = self._get_label_encoder_and_max(X[col])

            X.loc[:, col] = X[col].fillna(NAN_INT).map(self.label_encoders[i]).fillna(0).astype(int)

        self.is_fitted = True
        return X

In [3]:
path = "../input/web-ctr-prediction/"
train = pd.read_parquet(path + "train_sample_0.3_seed1119.parquet")
test = pd.read_parquet(path + "test.parquet")

## DeepFM

In [4]:
sparse_features = train.select_dtypes("object").columns.to_list()[1:]

In [5]:
lbe = LabelEncoder(10)
train[sparse_features] = lbe.fit_transform(train[sparse_features])
test[sparse_features] = lbe.transform(test[sparse_features])

for col in sparse_features:
    train[col] = train[col].astype(int)
    test[col] = test[col].astype(int)

train = train.fillna(0)
test = test.fillna(0)

In [6]:
dense_features = train.select_dtypes("int64").columns.to_list() + train.select_dtypes("float64").columns.to_list()
dense_features = dense_features[1:]

mms = MinMaxScaler(feature_range=(0, 1))
train[dense_features] = mms.fit_transform(train[dense_features])
test[dense_features] = mms.transform(test[dense_features])

In [7]:
def reduce_mem_usage(df: pd.DataFrame, verbose: bool = True) -> pd.DataFrame:
    """
    Iterate through all the columns of a dataframe and modify the data type to reduce memory usage.
    """
    numerics = ["int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024**2

    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2

    if verbose:
        print(f"Mem. usage decreased to {end_mem:5.2f} Mb ({100 * (start_mem - end_mem) / start_mem:.1f}% reduction)")

    return df


train = reduce_mem_usage(train)
test = reduce_mem_usage(test)

Mem. usage decreased to 1035.50 Mb (73.5% reduction)
Mem. usage decreased to 372.23 Mb (73.1% reduction)


In [8]:
dnn_feature_columns = [
    SparseFeat(feat, vocabulary_size=train[feat].nunique(), embedding_dim=4) for i, feat in enumerate(sparse_features)
] + [DenseFeat(feat, 1) for feat in dense_features]

linear_feature_columns = [
    SparseFeat(feat, vocabulary_size=train[feat].nunique(), embedding_dim=4) for i, feat in enumerate(sparse_features)
] + [DenseFeat(feat, 1) for feat in dense_features]

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=1119)

X = train.drop(columns=["Click"])
y = train["Click"]

oof_preds = np.zeros((X.shape[0],))
nn_preds = np.zeros((test.shape[0],))
scores = []

for fold, (train_idx, valid_idx) in enumerate(iterable=kfold.split(X=X, y=y), start=1):
    train_fold, valid_fold = train.iloc[train_idx], train.iloc[valid_idx]

    train_model_input = {name: train_fold[name] for name in feature_names}
    valid_model_input = {name: valid_fold[name] for name in feature_names}
    test_model_input = {name: test[name] for name in feature_names}

    model = DeepFM(
        dnn_feature_columns=dnn_feature_columns,
        linear_feature_columns=linear_feature_columns,
        device=device,
        dnn_activation="prelu",
        dnn_use_bn=True,
        l2_reg_linear=0,
        l2_reg_embedding=0,
    )

    model.compile(
        MADGRAD(model.parameters(), lr=0.0001),
        "binary_crossentropy",
        metrics=["binary_crossentropy", "auc"],
    )

    es = EarlyStopping(
        monitor="val_binary_crossentropy",
        min_delta=0,
        verbose=1,
        patience=2,
        mode="min",
    )

    history = model.fit(
        train_model_input,
        train_fold["Click"].values,
        batch_size=4096,
        epochs=100,
        verbose=1,
        validation_data=(valid_model_input, valid_fold["Click"].values),
        callbacks=[es],
    )
    val_ans = model.predict(valid_model_input, batch_size=64).flatten()
    oof_preds[valid_idx] = val_ans
    nn_preds += model.predict(test_model_input, batch_size=64).flatten() / 5

    print(f"Fold{fold} test LogLoss: {log_loss(valid_fold['Click'].values, val_ans):.4f}")
    print(f"Fold{fold} test AUC: {roc_auc_score(valid_fold['Click'].values, val_ans):.4f}")
    
    scores.append(roc_auc_score(valid_fold["Click"].values, val_ans))
    del train_fold, valid_fold, model
    gc.collect()


print(np.mean(scores))


cuda:0
Train on 9984415 samples, validate on 2496104 samples, 2438 steps per epoch


2438it [02:24, 16.86it/s]


Epoch 1/20
156s - loss:  0.6318 - binary_crossentropy:  0.6318 - auc:  0.6902 - val_binary_crossentropy:  0.6273 - val_auc:  0.7011


2438it [02:24, 16.93it/s]


Epoch 2/20
155s - loss:  0.6278 - binary_crossentropy:  0.6278 - auc:  0.6982 - val_binary_crossentropy:  0.6231 - val_auc:  0.7055


...

2438it [02:30, 16.16it/s]


Epoch 15/20
162s - loss:  0.6036 - binary_crossentropy:  0.6036 - auc:  0.7296 - val_binary_crossentropy:  0.6070 - val_auc:  0.7283
Epoch 00015: early stopping
Fold4 test LogLoss: 0.6070
Fold4 test AUC: 0.7283
cuda:0
Train on 9984416 samples, validate on 2496103 samples, 2438 steps per epoch


2438it [02:30, 16.25it/s]


Epoch 1/20
161s - loss:  0.6314 - binary_crossentropy:  0.6314 - auc:  0.6907 - val_binary_crossentropy:  0.6234 - val_auc:  0.7022


1589it [01:36, 19.22it/s]

In [ ]:
del test

## Ensemble

In [ ]:
submission = pd.read_csv(path + "sample_submission.csv")
submission["Click"] = nn_preds
submission.to_csv("../output/5fold-ctr-deepfm.csv", index=False)

In [ ]:
submission.head()